## **Packages**

In [1]:
# packages
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from datetime import date, datetime
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit
from hdfs import InsecureClient
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt #그래프 패키지 모듈 등록
%matplotlib inline 

In [2]:
# matplotlib 한글폰트 
import platform

from matplotlib import font_manager, rc
plt.rcParams['axes.unicode_minus'] = False

if platform.system() == 'Darwin':  # 맥OS 
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':  # 윈도우
    path = "c:/Windows/Fonts/malgun.ttf"
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    rc('font', family='D2Coding')
       
# rc('font', family='NanumGothic') # 나눔폰트 사용시

## **DW에 있는 Data 사용하기 위한 Hdfs 클라이언트 객체 생성**

In [3]:
client = InsecureClient('http://localhost:9870', user='lab09')

## **Data 불러오기**
- 관광지 평점 데이터 = tour_ratings.json

In [4]:
keywords = spark.read.json('/raw_data/tour_keywords.json', encoding='utf-8')

In [5]:
keywords.show()

+-------------+-------------+-------------+--------------------------------+-----+
|Top_Keyword_1|Top_Keyword_2|Top_Keyword_3|               Tourist_Spot_Name|index|
+-------------+-------------+-------------+--------------------------------+-----+
|    adventure|       joyful|          art|                    (주)교보문고|    1|
|    tradition|     cultural|    adventure|            1.21 무장공비 침투로|    2|
|       joyful|    adventure|          art|           2024 코리아그랜드세일|    3|
|    adventure|       joyful|     cultural|   24시 마린낚시(전곡항 24 마...|    4|
|          art|     cultural|       joyful|                        93뮤지엄|    5|
|       joyful|          art|     cultural|                     CICA 미술관|    6|
|          art|    tradition|       joyful|    C아트뮤지엄(숲속의 미술공원)|    7|
|    tradition|       nature|     cultural|                    DMZ 마루캠핑|    8|
|     cultural|       nature|    adventure|                    DMZ 생생누리|    9|
|    tradition|       joyful|    adventure|                

## **✅ 확인**

In [6]:
# 관광지 정보 데이터 불러오기
travel_data = spark.read.json('/dw_data/travel_data_preprocessed.json', encoding='utf-8')
travel_data_pd = travel_data.toPandas()
travel_data_pd.head()
travel_data_pd.info()

,index,개요,개장기간,공연시간,관광지 분류,관람소요시간,명칭,분류,상세정보,쉬는날,...,이용요금,입장료,전화번호,주소,주차시설,주차요금,지역,체험안내,행사시작일,행사종료일
0,1,"교보문고 광화문점은 국민교육 진흥의 실천적 구현, 독서인구 저변 확대를 통한 국민정...",None,None,문화시설,None,(주)교보문고,문화시설,None,설/추석 당일,...,무료,None,None,서울특별시 종로구 종로 1(종로1가),주차 가능,유료 (회원여부/구매금액에 따라 무료주차헤택 적용),서울,None,NaN,NaN
1,2,※ 민통초소에 신분증 제시하여 출입 가능 (25인 이상 출입시 일주일 전 신청서 제...,None,None,인문관광지,None,1.21 무장공비 침투로,역사관광지,화장실:있음\n,매주 화요일,...,None,None,None,경기도 연천군 장남면 반정리 294,None,None,경기도,None,NaN,NaN
2,3,코리아그랜드세일은 관광과 한류가 융복합된 외국인 대상의 한국 대표 쇼핑문화관광축제로...,None,None,축제공연행사,None,2024 코리아그랜드세일,공연행사,행사소개:코리아그랜드세일은 관광과 한류가 융복합된 외국인 대상의 한국 대표 쇼핑문화...,None,...,무료,None,070-7787-4242,서울특별시 종로구 인사동5길 29 태화빌딩802호,None,None,서울,None,2.0240111E7,2.0240229E7
3,4,"경기도 화성시 서신면에 위치해 있는 24시 마린 낚시는 주변에 전곡항, 탄도항, 제...",None,None,레포츠,None,24시 마린낚시(전곡항 24 마린낚시),레포츠,"포인트안내:\n이용요금:선박별 상이 홈페이지 참조\n보유장비: 22인승, 21인승,...",None,...,None,None,None,경기도 화성시 서신면 전곡항로 317,None,None,경기도,None,NaN,NaN
4,5,그림을 열망하던 청년은 첫 월급 3000원을 쪼개 그림 한 점을 구입했다. 당시 가...,None,None,문화시설,None,93뮤지엄,문화시설,None,"매주 월요일(단,월요일이 공휴일인 경우 정상 개관,해당주 화요일 휴관)",...,기간에따라 변동,None,None,경기도 파주시 탄현면 헤이리마을길 59-58헤이리예술마을 G-31,주차가능,None,경기도,None,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3504 entries, 0 to 3503
Data columns (total 25 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   index          3504 non-null   int64 
 1   개요             3501 non-null   object
 2   개장기간           577 non-null    object
 3   공연시간           30 non-null     object
 4   관광지 분류         3504 non-null   object
 5   관람소요시간         240 non-null    object
 6   명칭             3504 non-null   object
 7   분류             3504 non-null   object
 8   상세정보           3038 non-null   object
 9   쉬는날            2304 non-null   object
 10  애완동물 동반 가능 여부  3504 non-null   object
 11  우편번호           3400 non-null   object
 12  유모차 대여 여부      3504 non-null   object
 13  이용시간           2678 non-null   object
 14  이용시기           11 non-null     object
 15  이용요금           670 non-null    object
 16  입장료            22 non-null     object
 17  전화번호           104 non-null    object
 18  주소             3502 non-null

In [7]:
# keywords data pandas
keywords_pd = keywords.toPandas()
keywords_pd.head()
keywords_pd.info()

,Top_Keyword_1,Top_Keyword_2,Top_Keyword_3,Tourist_Spot_Name,index
0,adventure,joyful,art,(주)교보문고,1
1,tradition,cultural,adventure,1.21 무장공비 침투로,2
2,joyful,adventure,art,2024 코리아그랜드세일,3
3,adventure,joyful,cultural,24시 마린낚시(전곡항 24 마린낚시),4
4,art,cultural,joyful,93뮤지엄,5


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3504 entries, 0 to 3503
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Top_Keyword_1      3504 non-null   object
 1   Top_Keyword_2      3504 non-null   object
 2   Top_Keyword_3      3504 non-null   object
 3   Tourist_Spot_Name  3504 non-null   object
 4   index              3504 non-null   int64 
dtypes: int64(1), object(4)
memory usage: 137.0+ KB


In [8]:
mismatched_rows = travel_data_pd[~(travel_data_pd[['index', '명칭']]
                                   .apply(tuple, axis=1)
                                   .isin(keywords_pd[['index', 'Tourist_Spot_Name']]
                                         .apply(tuple, axis=1)))]

In [9]:
mismatched_rows

,index,개요,개장기간,공연시간,관광지 분류,관람소요시간,명칭,분류,상세정보,쉬는날,...,이용요금,입장료,전화번호,주소,주차시설,주차요금,지역,체험안내,행사시작일,행사종료일


In [10]:
keywords_pd[(keywords_pd['index'] >= 2095) & (keywords_pd['index'] < 2196)]

,Top_Keyword_1,Top_Keyword_2,Top_Keyword_3,Tourist_Spot_Name,index
2094,nature,cultural,joyful,노을공원(인천),2095
2095,nature,adventure,art,노을캠핑장,2096
2096,cultural,tradition,nature,노작홍사용문학관,2097
2097,nature,adventure,tradition,노적봉공원,2098
2098,adventure,joyful,cultural,노진낚시터,2099
...,...,...,...,...,...
2190,tradition,nature,cultural,돌담캠핑가든,2191
2191,adventure,joyful,art,돌체소극장,2192
2192,art,joyful,cultural,동국대학교 박물관,2193
2193,adventure,tradition,nature,동대문역사문화공원,2194


In [11]:
travel_data_pd[['index', '명칭']][(travel_data_pd['index'] >= 2095) & (travel_data_pd['index'] < 2196)]

,index,명칭
2094,2095,노을공원(인천)
2095,2096,노을캠핑장
2096,2097,노작홍사용문학관
2097,2098,노적봉공원
2098,2099,노진낚시터
...,...,...
2190,2191,돌담캠핑가든
2191,2192,돌체소극장
2192,2193,동국대학교 박물관
2193,2194,동대문역사문화공원


## **👍 파일 저장**

In [12]:
# hdfs에 저장
keywords.coalesce(1).write.format('json').save('/dw_data/tour_keywords.json')